# Introduction
This notebook generates the trained model for https://www.kaggle.com/hdsk38/pytorch-starter-inference-efficientnet, which should give Public Score 0.003. Many codes are forked from https://www.kaggle.com/rhtsingh/pytorch-training-inference-efficientnet-baseline. Thank you [
torch](https://www.kaggle.com/rhtsingh)!

Since the train data is big (1,580,470 images!!), I filtered the train data by selecting 10 images from each of the top 30,000 classes (out of 81,313 classes). Only 3 epochs were implemented in order to finish the training within a couple of hours. I hope this notebook will help you to join this competition :) !

In [ ]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/ > /dev/null # no output
!pip install torch_optimizer --no-index --find-links=file:///kaggle/input/torch-optimizer/torch_optimizer

In [ ]:
import os
import gc
gc.enable()
import sys
import math
import json
import time
import random
from glob import glob
from datetime import datetime

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
from sklearn.preprocessing import LabelEncoder

import torch
import torchvision
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm import tqdm

import efficientnet_pytorch

import torch_optimizer as optim
import albumentations as A

import sklearn

import warnings
warnings.filterwarnings("ignore")

In [ ]:
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
BATCH_SIZE = 64
NUM_WORKERS = multiprocessing.cpu_count()
MAX_STEPS_PER_EPOCH = 15000
NUM_EPOCHS = 3
LOG_FREQ = 10
NUM_TOP_PREDICTS = 1

In [ ]:
train = pd.read_csv('../input/landmark-recognition-2021/train.csv')
train_dir = '../input/landmark-recognition-2021/train/'

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame, image_dir:str, mode: str):
        self.df = dataframe
        self.mode = mode
        self.image_dir = image_dir
        
        transforms_list = []
        if self.mode == 'train':
            transforms_list = [
                transforms.Resize((64,64)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomResizedCrop(64),
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
                                            scale=(0.8, 1.2), shear=15,
                                            resample=Image.BILINEAR)
                ]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ]
        else:
            transforms_list.extend([
                # Keep this resize same as train
                transforms.Resize((64,64)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ])
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index: int):
        image_id = self.df.iloc[index].id
        image_path = f"{self.image_dir}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.jpg"
        image = Image.open(image_path)
        image = self.transforms(image)

        if self.mode == 'test':
            return {'image':image}
        else:
            return {'image':image, 
                    'target':self.df.iloc[index].landmark_id}

    def __len__(self) -> int:
        return self.df.shape[0]

In [ ]:
def load_data(train, train_dir):
    print('Original train_df size:', train.shape)     
    counts = train.landmark_id.value_counts()
    
    num_samples=10
    num_classes = 30000
    print(f'Select train data from top {num_classes} classes, {num_samples} samples of each class')
    x=[]
    y=[]
    
    for k,landmark_id in enumerate(tqdm(counts.index[:num_classes])):
        x.extend(train[train['landmark_id']==landmark_id]['id'].sample(n=num_samples, random_state=1,replace=True).values)
        y.extend([landmark_id]*num_samples)
    train=pd.DataFrame({'id':x,'landmark_id':y})
    print('Filtered train_df size:', train.shape)

    #Encode target labels with value between 0 and n_classes-1
    label_encoder = LabelEncoder() 
    label_encoder.fit(train.landmark_id.values)
    
    #Check if the number of the classes are ok
    assert len(label_encoder.classes_) == num_classes 

    train.landmark_id = label_encoder.transform(train.landmark_id)

    train_dataset = ImageDataset(train, train_dir, mode='train')

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS,
                              drop_last=True)

    return train_loader, label_encoder, num_classes

In [ ]:
def radam(parameters, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
    if isinstance(betas, str):
        betas = eval(betas)
    return optim.RAdam(parameters,
                      lr=lr,
                      betas=betas,
                      eps=eps,
                      weight_decay=weight_decay)

In [ ]:
class AverageMeter:
    ''' Computes and stores the average and current value '''
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1) -> None:
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def GAP(predicts: torch.Tensor, confs: torch.Tensor, targets: torch.Tensor) -> float:
    ''' Simplified GAP@1 metric: only one prediction per sample is supported '''
    assert len(predicts.shape) == 1
    assert len(confs.shape) == 1
    assert len(targets.shape) == 1
    assert predicts.shape == confs.shape and confs.shape == targets.shape

    _, indices = torch.sort(confs, descending=True)

    confs = confs.cpu().numpy()
    predicts = predicts[indices].cpu().numpy()
    targets = targets[indices].cpu().numpy()

    res, true_pos = 0.0, 0

    for i, (c, p, t) in enumerate(zip(confs, predicts, targets)):
        rel = int(p == t)
        true_pos += rel

        res += true_pos / (i + 1) * rel

    res /= targets.shape[0] # FIXME: incorrect, not all test images depict landmarks
    return res

In [ ]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}')
        pretrained_file = glob(f'../input/efficientnet-pytorch/efficientnet-b{self.depth}*')[0]
        checkpoint = torch.load(pretrained_file)
        self.base.load_state_dict(checkpoint)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
        
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x

In [ ]:
def train_step(train_loader, 
          model, 
          criterion, 
          optimizer,
          epoch, 
          lr_scheduler):
    print(f'epoch {epoch}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    avg_score = AverageMeter()

    model.train()
    num_steps = min(len(train_loader), MAX_STEPS_PER_EPOCH)

    print(f'total batches: {num_steps}')

    end = time.time()
    lr = None

    for i, data in enumerate(train_loader):
        input_ = data['image']
        target = data['target']
        batch_size, _, _, _ = input_.shape
        
        output = model(input_.cuda())
        loss = criterion(output, target.cuda())
        confs, predicts = torch.max(output.detach(), dim=1)
        avg_score.update(GAP(predicts, confs, target))
        losses.update(loss.data.item(), input_.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        
        batch_time.update(time.time() - end)
        end = time.time()

        if i % LOG_FREQ == 0:
            print(f'{epoch} [{i}/{num_steps}]\t'
                    f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
                    f'GAP {avg_score.val:.4f} ({avg_score.avg:.4f})'
                    + str(lr))

    print(f' * average GAP on train {avg_score.avg:.4f}')

# Train model

In [ ]:
if __name__ == '__main__':
    print("A")
    train_loader, label_encoder, num_classes = load_data(train, train_dir)
    
    print("B")
    model = EfficientNetEncoderHead(depth=0, num_classes=num_classes)
    
    print("C")
    model.cuda()
    
    print("D")
    criterion = nn.CrossEntropyLoss()
    
    print("E")
    optimizer = radam(model.parameters(), lr=1e-3, betas=(0.9,0.999), eps=1e-3, weight_decay=1e-4)
    
    print("F")
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*NUM_EPOCHS, eta_min=1e-6)
    
    print("G")
    for epoch in range(1, NUM_EPOCHS + 1):
        print('-' * 50)
        train_step(train_loader, model, criterion, optimizer, epoch, scheduler)

# Save classes of the label_encoder

In [ ]:
np.save('classes.npy', label_encoder.classes_)

# Save model weights

In [ ]:
torch.save(model.state_dict(), "new_weight_efficientnet.pth")